# Create Jupyter Book

## 1. Installation

To install the Jupyter Book command-line interface (CLI), use `pip`!

In [ ]:
import sys

#install jupyter-book
cmd = f'{sys.executable} -m pip show jupyter-book'
cmdOutput = !{cmd}
if len(cmdOutput) > 1 and '0.6.4' in cmdOutput[1]:
    print('Jupyter-book required version is already installed!')
else:
    !pip install jupyter-book

## 2. Create a new book

Create a book using your own notebooks and markdown pages:

<span style="color:red">Note:</span> Notebook and markdown filenames cannot contain spaces

In [ ]:
import os, re, shutil

overwrite = False
book_name = input('Please provide the path where the book needs to be saved along with the book name ex-> D:\Book1: ') 

if (os.path.exists(book_name)):
    new_book_name = input('A folder named ' + book_name + ' already exists. Enter a new name or the same name to overwrite the existing folder.\n')
    if book_name == new_book_name:
        overwrite = True
    book_name = new_book_name

content_folder = input('Please provide the path to your folder containing notebooks and markdown files: ')

while (not os.path.exists(content_folder)):
    content_folder = input('Cannot find folder ' + content_folder + '. Please provide another path: ')
 
if overwrite:
    !jupyter-book create "$book_name" --content-folder "$content_folder" --overwrite
else:
    !jupyter-book create "$book_name" --content-folder "$content_folder"

In [ ]:
# Update toc file, book title and clean up the directores
tocFilePath = os.path.join(book_name, "_data", "toc.yml")
f = open(tocFilePath, "r")
title = ''
replacedString = ''
result = f.read()
f.close()
contentFolders = []

firstLevelUrls = re.findall(r'^(?:\s+$[\r\n]+)+(\- url: [a-zA-Z0-9\\.\s\-\/]+$[\r\n]+)', result, re.MULTILINE)
urls = re.findall(r'- url: [a-zA-Z0-9\\.\s\-\/]+$', result, re.MULTILINE)
headers = re.findall(r'- header: [a-zA-Z0-9\\.\s-]+$', result, re.MULTILINE)

try:
    if (firstLevelUrls or headers or urls):
        if (firstLevelUrls and len(firstLevelUrls) == 1):
            for url in firstLevelUrls:
                title = url[url.rindex(os.path.sep)+1:].rstrip()
                if (not headers):
                    markdownUrl = urls[len(urls) -1]
                    title = markdownUrl[markdownUrl.rindex(os.path.sep)+1:].rstrip()
                    replacedString = "\n- title: %s\n  url: /%s\n  not_numbered: true\n  expand_sections: true\n  sections:  %s" % (title, title, url)
                    result = result.replace(markdownUrl, '')
                else:
                    replacedString = "\n- title: %s\n  url: /%s\n  not_numbered: true\n" % (title, title)
                result = result.replace(url, replacedString)
        if (headers):
            for header in headers:
                title = header[10:].rstrip()
                contentFolders.append(title.lower())
                filtered = list(filter(lambda x: ("%s%s%s" % (os.path.sep, title.lower(), os.path.sep)) in x, urls))
                index = urls.index(filtered[len(filtered)-1])
                urlValue = urls[index][urls[index].rindex(os.path.sep)+1:].rstrip()
                replacedString = "\n- title: %s\n  url: /%s/%s\n  not_numbered: true\n  expand_sections: true\n  sections:  " % (title, title.lower(), urlValue)
                result = result.replace(header, replacedString)
                result = result.replace(urls[index], '')
                del urls[index]
        if (urls):
            for url in urls:
                title = url[url.rindex(os.path.sep)+1:].rstrip()
                urlValue = title
                if (len(contentFolders) > 0):
                    folders = url[7:].split(os.path.sep)
                    if (folders[len(folders)-2] in contentFolders):
                        parentFolder = contentFolders.index(folders[len(folders)-2])
                        urlValue = "%s/%s" % (contentFolders[parentFolder], title)
                replacedString = "\n  - title: %s\n    url: /%s" % (title, urlValue)
                result = result.replace(url, replacedString)
        fwrite = open(tocFilePath, "w")
        fwrite.write(result)
        fwrite.close()
    else:
        raise SystemExit(f'\n File Name contains unsupported-characters (ex: underscores) by Jupyter Book.\n')
    # Update the Book title in config file
    configFilePath = os.path.join(book_name, "_config.yml")
    f = open(configFilePath, "r")
    result = f.read()
    f.close()
    titleLine = re.search(r'title: [a-zA-Z0-9\\.\s\-\/]+$', result, re.MULTILINE).group()
    title = 'title: %s' % (os.path.splitext(os.path.basename(book_name))[0])
    result = result.replace(titleLine, title)
    fwrite = open(configFilePath, "w")
    fwrite.write(result)
    fwrite.close()
    # cleanup the directories
    with os.scandir(book_name) as root_dir:
        for path in root_dir:
            if path.is_file() and path.name not in ('_config.yml'):
                os.remove(path)
            if path.is_dir() and path.name not in ('_data', 'content'):
                shutil.rmtree(path)
except Exception as e:
    print(str(e))

## 3. Open your Book!
**Run the below cell and click on the link to view your book in Azure Data Studio.**

In [ ]:
import re, os
from IPython.display import *
if os.name == 'nt':
    bookPath = book_name.replace('\\', '\\\\')
    display(HTML("<h2><b><a href=\"command:bookTreeView.openBook?&quot;"+str(bookPath)+"&quot;\"><font size=\"3\">Click here to open your Book in ADS</font></a></b></h2>"))
else:
    display(HTML("<h2><b><a href=\"command:bookTreeView.openBook?&quot;"+str(book_name)+"&quot;\"><font size=\"3\">Click here to open your Book in ADS</font></a></b></h2>"))

<span style="color:red">**Note**: On clicking the above link, we create a temporary toc.yml file for your convenience.</span>

 Please update that file inside your book (located at: *YourbookPath*/_data/toc.yml) if you want to further customize your book following 
 instructions at https://jupyterbook.org/guide/01-5_tour.html#Table-of-Contents.


In [ ]:
display(HTML("<h1><b>That's it!</b></h1><br/><p>You are good to view your book in Azure Data Studio by clicking on the above link.</p>"))